<a href="https://colab.research.google.com/github/aubreymoore/crb-symbiont/blob/main/symbiont.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# symbiont

This Google Colab workbook analyses data stored in a Google Sheet named **CRB Gut Symbiont Experiment Working Sheet**.

Use **Ctrl+F9** to run all cells.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from google.colab import auth
import gspread
from google.auth import default

# Import data from Google Sheet

In [2]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
worksheet = gc.open('CRB Gut Symbiont Experiment Working Sheet').sheet1
df = pd.DataFrame(worksheet.get_all_records())
df['weight in mg'] = df['weight in mg'].replace('', np.nan)
df['weight in mg'] = df['weight in mg'].astype(np.float64)
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df.sort_values(by=['Date', 'Treatment'], ascending=[True,False], inplace=True) # to fix problem with order of treatment groups in plot legend
df

,weight in mg,Clutch ID,Treatment,Date,Week,Instar,Initials,Infection notes,general notes
0,NaN,Cl_644,untreated,2023-02-06,0,egg,LRC,,
1,NaN,Cl_644,untreated,2023-02-06,0,egg,LRC,,
2,NaN,Cl_644,untreated,2023-02-06,0,egg,LRC,,
3,NaN,Cl_644,untreated,2023-02-06,0,egg,LRC,,
4,NaN,Cl_644,untreated,2023-02-06,0,egg,LRC,,
...,...,...,...,...,...,...,...,...,...
1540,1939.2,CL_663_3,treated,2023-05-08,12,2,AM,,
1541,1166.5,CL_663_4,treated,2023-05-08,12,2,AM,,
1542,742.5,CL_663_5,treated,2023-05-08,12,2,AM,,
1543,1270.9,CL_663_6,treated,2023-05-08,12,2,AM,,


# Plot larval mass vs time

In [3]:
px.scatter(df, title = 'Mass of CRB larvae fed sterilized coconut material with and without addition of frass', 
           x='Week', y="weight in mg", color='Treatment', height=1000, width=800, trendline='lowess', 
           opacity=0.4)

# Plot survival curves

In [6]:
df_larvae = df[df['general notes'].str.contains('dead') == False]
df_larvae = df_larvae.query( 'Week >= 3 and Instar in [1,2,3]' ) # it takes as long as 3 weeks fro eggs to hatch, so we start counting larvae at 3 weeks post-treatment
gb = df_larvae.groupby(['Week','Treatment']).size().reset_index()
gb = gb.rename({0:'number of larvae remaining alive'}, axis=1)
gb.sort_values(by=['Week', 'Treatment'], ascending=[True,False], inplace=True) # to fix problem with order of treatment groups in plot legend
px.scatter(gb, title = 'Survival curves for CRB larvae fed sterilized coconut material with and without addition of frass', 
           x='Week', y='number of larvae remaining alive', color='Treatment', height=1000, width=800, trendline='lowess', trendline_options=dict(frac=0.2), opacity=0.4)

In [ ]:
df_alive = df[df['general notes'].str.contains('dead') == False]
df_alive.groupby(['Week','Treatment','Instar']).size().reset_index().rename({0:'count'}, axis=1)

,Week,Treatment,Instar,count
0,0,treated,egg,150
1,0,untreated,egg,150
2,1,treated,egg,150
3,1,untreated,egg,150
4,2,treated,1,42
5,2,treated,egg,40
6,2,untreated,1,34
7,2,untreated,egg,65
8,3,treated,1,74
9,3,untreated,1,76


In [ ]:
df_alive.query('Instar == ""')

,weight in mg,Clutch ID,Treatment,Date,Week,Instar,Initials,Infection notes,general notes
1226,NaN,Cl_659,treated,2023-03-28,6,,LRC,possible infection,
1227,NaN,Cl_660,treated,2023-03-28,6,,LRC,possible infection,
1367,NaN,Cl_659,treated,2023-04-10,8,,LRC,,
1368,NaN,Cl_660,treated,2023-04-10,8,,LRC,,
